* Take max of frames as in 2015 paper
* Start states are not stored, so replay of startt is techn. false

* [x] Decide whether to generate video or gif -- gif
* [x] Generate vid/gif
* [/] Decide how to store results
* [x] Extra record when max score beats prev max score
* [x] Eval round during training
* [x] Do random moves before fire
* [ ] Decide how many ran moves to do
* [x] Set huber loss function
* [ ] Tensor board
* [ ] Visualization
* [ ] 1, 2, 4 freq
* [ ] Initialization
* [ ] DDQN


config_proto.intra_op_parallelism_threads = 1
config_proto.inter_op_parallelism_threads = 1

NUM_PARALLEL_EXEC_UNITS = 4
config = tf.ConfigProto(intra_op_parallelism_threads=NUM_PARALLEL_EXEC_UNITS, inter_op_parallelism_threads=2,
                       allow_soft_placement=True, device_count={'CPU': NUM_PARALLEL_EXEC_UNITS})
session = tf.Session(config=config)
K.set_session(session)
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["KMP_BLOCKTIME"] = "30"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"] = "granularity=fine,verbose,compact,1,0"

__NIPS Paper__
* DQN
* 10 million frames training
* avg r for eps=0.05 (168), max r (225)
* epoch = 50,000 minibatch updates (i.e. steps) ~ 30min training (looks from plots like the do 5mm frames...)
* no target model
* no no-ops
* no treat life loss as end of episode
* no huber loss
* replay start size 100
* update freq 1
* LEARNING_RATE = .0002
* DISCOUNT = .95
* RMS_DECAY = .99 # (Rho)
* RMS_EPSILON = 1e-6
* MOMENTUM = 0
    
__Augmentations__
* Target network -- DONE
* Double DQN (in update: main to use best action, but target to select Q), run too and compare
* Huber loss - DONE
* Remove reward clipping -- DONE
* Bigger network
* Lower # of no_op_steps
* In tensorflow use tf.variance_scaling_initializer with scale = 2
* Make sure you are updating the networks in the right frequency: The paper says that the target network update frequency is "measured in the number of parameter updates" whereas in the code it is measured in the number of action choices/frames the agent sees.
* Batch AC3 algorithm?
* Use OpenAI benchmarking tool

__Material Notes__
* Boyuan implementation of target net looks wrong, it's only used for action selection, not to generate actual Q targets.
* Further, whenever we call load_model(remember, we needed it for the target network), we will need to pass custom_objects={'huber_loss': huber_loss
* Boyuan "If agent is dead, set the flag back to false, but keep the history unchanged, to avoid to see the ball up in the sky" 
* next_q_vals = lasagne.layers.get_output(self.next_l_out, next_states / input_scale)

In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
import pickle
import random
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import clone_model
import gym
import os

import sys
sys.path.append('../script')
from utils import preprocess_frame_v1, choose_action, clip_reward, get_lin_anneal_eps, \
    plot_state, frames_to_mp4
from atari_model import atari_model, fit_batch, fit_batch_DQN
from replay_memory import ReplayMemory

In [2]:
# Set parameters
total_train_len = 50000      # Total no. of episodes to train over
max_episode_len = None       # Max no. of frames agent is allowed to see per episode
state_len = 4                # No. of stacked frames that comprise a state
train_interval = 2           # Every x actions a gradient descend step is performed
tgt_update_interval = 10000  # ...
eps_init = 1                 # Initial eps in eps-greedy exploration 
eps_final = 0.1              # Final eps in eps-greedy exploration
eps_final_frame = 1000000    # No. of frame over which eps is linearly annealed to final eps
burnin_sz = 100            # Replay mem. burn-in: random policy is run for this many frames, training starts after
replay_mem_sz = 1000000      # Max no. of frames cached in replay memory
batch_sz = 32                # No. of training cases (sample from replay mem.) for each SGD update
disc_rate = 0.99             # Q-learning discount factor (gamma)
seed = 1234

In [3]:
# Initialize Atari environment
env = gym.make('BreakoutDeterministic-v4')

# Set seeds
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
env.seed(seed)
env.action_space.seed(seed)

[1234]

In [13]:
# Storage 
model_name = 'DQN_2freq'
os.mkdir(f'../model/{model_name}')
os.mkdir(f'../model/{model_name}/record')
os.mkdir(f'../model/{model_name}/model')

# Initialize new Q-net model and replay memory
model = atari_model(env.action_space.n)
model_tgt = clone_model(model)
model_tgt.set_weights(model.get_weights())

replay_mem = ReplayMemory(replay_mem_sz, state_len)
frame_num = 0
max_episode_reward = 0
episode_start = 0

In [14]:
def play_episode(model, env):
    # Start a new game (episode)
    init_frame = env.reset()
    new_life = True
    game_over = False
    epsilon = 0.001
    
    # Keep track of episode figures
    episode_reward = 0
    episode_frames = [init_frame]
    
    # Play episode until game over
    while not game_over:
        if new_life:
            # Start a new life in the game
            frame, _, _, info = env.step(1)  # Fire to start playing
            episode_frames.append(frame)
#             for _ in range(random.randint(1, 5)):
#                 frame, _, game_over, info = env.step(env.action_space.sample())
#                 episode_frames.append(frame)
            
            lives = info['ale.lives']
            frame = preprocess_frame_v1(frame)
            state = np.stack(state_len * [frame], axis=2)
        else:
            state = np.append(state[:, :, 1:], frame[:, :, None], axis=2)
        
        action = choose_action(env, model, state, epsilon)
        frame, reward, game_over, info = env.step(action)
        episode_frames.append(frame)
        
        # Process env outputs
        frame = preprocess_frame_v1(frame)
        new_life = info['ale.lives'] < lives 
        lives = info['ale.lives']
         
        # Increase 
        episode_reward += reward
    
    return episode_frames, episode_reward 

In [ ]:
# Start the fun
for episode_num in range(episode_start, total_train_len):
    # Start a new game (episode)
    init_frame = env.reset()
    new_life = True
    game_over = False
    
    # Keep track of episode figures
    episode_reward = 0
    episode_frames = [init_frame]
    
    # Play episode until game over
    while not game_over:
        if new_life:
            # Start a new life in the game
            frame, _, _, _ = env.step(1)  # Fire to start playing
            episode_frames.append(frame)
            for _ in range(random.randint(1, 15)):
                frame, _, game_over, info = env.step(env.action_space.sample())
                episode_frames.append(frame)
            
            lives = info['ale.lives']
            frame = preprocess_frame_v1(frame)
            state = np.stack(state_len * [frame], axis=2)
        else:
            state = np.append(state[:, :, 1:], frame[:, :, None], axis=2)
        
        # Select action
        burnin_done = frame_num > burnin_sz
        eps = get_lin_anneal_eps(frame_num - burnin_sz, eps_init, eps_final, eps_final_frame)
        eps = eps if burnin_done else 1
        action = choose_action(env, model, state, eps)

        # Take action
        frame, reward, game_over, info = env.step(action)
        episode_frames.append(frame)
        
        # Process env outputs
        frame = preprocess_frame_v1(frame)
#         reward = clip_reward(reward)
        new_life = info['ale.lives'] < lives 
        lives = info['ale.lives']
        
        # Add new transition to replay memory
        transition = (action, reward, game_over or new_life, new_life, frame)    # inspect game + life
        replay_mem.store_memory(transition)
                   
        # After burn-in period start updating models
        if burnin_done:
            # Train model every train_interval
            if frame_num % train_interval == 0:
                mini_batch = replay_mem.get_sample(batch_sz)
                fit_batch_DQN(model, model_tgt, disc_rate, *mini_batch)
            
            # Update target model
            if frame_num % tgt_update_interval == 0:  #
                model_tgt.set_weights(model.get_weights())
         
        # Increase 
        frame_num += 1
        episode_reward += reward
    
    # Log every episode
    with open(f'../model/{model_name}/log.txt', 'a') as f:
        print(
            f'{datetime.utcnow()}, episode: {episode_num}, frame: {frame_num},'
            + f' episode_len: {len(episode_frames)}, score: {episode_reward}', 
            file=f
        )

    if episode_num % 100 == 0:
        # Store train results
        frames_to_mp4(f'../model/{model_name}/record/record_{episode_num}_train_{episode_reward}.mp4', episode_frames)
        model.save(f'../model/{model_name}/model/model_{episode_num}.h5')

        # Store eval results
        eval_frame, eval_reward = play_episode(model, env)
        frames_to_mp4(f'../model/{model_name}/record/record_{episode_num}_eval_{eval_reward}.mp4', eval_frame)
        
        # Store replay memory
        with open(f'../model/{model_name}/replay_mem.pkl', 'wb') as f:
            pickle.dump(replay_mem, f)
    
    elif episode_reward > max_episode_reward:
        # Store new max reward results
        frames_to_mp4(f'../model/{model_name}/record/record_{episode_num}_train_x_{episode_reward}.mp4', episode_frames)
        max_episode_reward = episode_reward

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

...

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima